In [1]:
1+1

2

In [2]:
%pwd

'/Users/suyash/Desktop/End to End/research'

In [3]:
import os
os.chdir("../")
%pwd

'/Users/suyash/Desktop/End to End'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    random_state: int
    n_jobs: int
    target_column: str 

In [5]:
from src.IPL_Predictor.constants import *
from src.IPL_Predictor.utils.common import read_yaml, create_directories
from src.IPL_Predictor.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            train_data_path=Path(config.train_data_path),
            test_data_path=Path(config.test_data_path),
            model_name=config.model_name,
            n_estimators=self.params.RandomForest.n_estimators,
            max_depth=self.params.RandomForest.max_depth,
            random_state=self.params.RandomForest.random_state,
            n_jobs=self.params.RandomForest.n_jobs,
            target_column=self.schema.TARGET_COLUMN,
        )

        return model_trainer_config

In [7]:
from sklearn.ensemble import RandomForestClassifier
import joblib
import pandas as pd
from src.IPL_Predictor import logger

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        # Load data with debug logging
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        # Log available columns
        logger.info(f"Available columns in training data: {train_data.columns.tolist()}")
        logger.info(f"Target column from config: {self.config.target_column}")
        
        # Check if we need to rename any columns or handle missing target
        if self.config.target_column not in train_data.columns:
            # Assuming the target column might be named differently
            available_columns = train_data.columns.tolist()
            logger.info(f"Target column '{self.config.target_column}' not found.")
            logger.info(f"Available columns: {available_columns}")
            raise ValueError(f"Target column '{self.config.target_column}' not found in training data. Available columns: {available_columns}")

        # Continue with model training if target exists
        X_train = train_data.drop(self.config.target_column, axis=1)
        y_train = train_data[self.config.target_column]
        X_test = test_data.drop(self.config.target_column, axis=1)
        y_test = test_data[self.config.target_column]

        rf = RandomForestClassifier(
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            random_state=self.config.random_state,
            n_jobs=self.config.n_jobs
        )

        rf.fit(X_train, y_train)
        
        # Create directory if it doesn't exist
        os.makedirs(self.config.root_dir, exist_ok=True)
        joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name))
        
        return rf

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model = model_trainer.train()
except Exception as e:
    raise e

[2025-09-24 12:56:23,918: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-24 12:56:23,920: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-24 12:56:23,921: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-24 12:56:23,921: INFO: common: created directory at : artifacts]
[2025-09-24 12:56:23,921: INFO: common: created directory at : artifacts/model_trainer]
[2025-09-24 12:56:24,415: INFO: 3552706665: Available columns in training data: ['target_runs', 'current_score', 'wickets', 'crr', 'rrr', 'batter_total_runs', 'batter_strikerate', 'non_striker_runs', 'non_striker_strikerate', 'city_Ahmedabad', 'city_Bangalore', 'city_Bengaluru', 'city_Bloemfontein', 'city_Cape Town', 'city_Centurion', 'city_Chandigarh', 'city_Chennai', 'city_Cuttack', 'city_Delhi', 'city_Dharamsala', 'city_Dubai', 'city_Durban', 'city_East London', 'city_Guwahati', 'city_Hyderabad', 'city_Indore', 'city_Jaipur', 'city_Johannesburg', 'city_Kanpur', 'city_Ki